The following are three different implementations of Linear Regression to demonstrate basic principles such as Gradient Descent, mini-batching in both numpy and Tensorflow.

https://github.com/clausherther/public/tree/master/machine_learning/linear_regression